In [71]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
%autoreload 1

In [73]:
%aimport event_selector
%aimport looper_utils
%aimport make_plots

In [74]:
import awkward as ak
import uproot
import numpy as np
from condor_utils import make_htcondor_cluster
from yahist import Hist1D, Hist2D

## Skim Parameters

In [75]:
year = '2018'
skim_version = 'v4'  #currently available x3 or v4

## Load data

In [77]:
%%bash
ls metadata/

chunklist_2016_v4.json
chunklist_2017_v4.json
chunklist_2018_v4.json
chunklist.json
totalWeights_2016_v4.json
totalWeights_2017_v4.json
totalWeights_2018_v4.json
totalWeights.json
xsection_2016_v4.json
xsection_2017_v4.json
xsection_2018_v4.json
xsection.json


In [78]:
import json
with open('./metadata/chunklist_'+year+'_'+skim_version+'.json') as f:
    chunks = json.load(f) 
chunks.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'ZH', 'VH', 'signal'])

## Process data

In [79]:
def process(args):
    
    import event_selector
    import make_plots
    
    obj_list = ["electron", "muon", "tau", "photon", "jet","others"]
   
    isData = "EGamma" in args[0] or "DoubleEG" in args[0]
    events = event_selector.prepare_inputs(args, obj_list, isData=isData)
    
    tot_w = ak.sum(events.genWeight)
    mgg = events.ggMass
    
    ## need to change gHidx
    gHidx = event_selector.get_gHidx(args) 
    g_Hidx = gHidx.gHidx
    mask_diphoton = event_selector.select_photon_byEvent(events.Photon, g_Hidx, mgg)
    
    mask_photon = event_selector.select_photon(events.Photon, g_Hidx, mgg)
    photons_selected = events.Photon[mask_photon] 
    nPho = ak.num(photons_selected)
    
    mask_tau = event_selector.select_tau(events.Tau, "all", isTight=False)
    ## clean by dR wrt photons
    mask_tau = mask_tau & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau = ak.num(events.Tau[mask_tau])
    
    mask_ele = event_selector.select_electron(events.Electron, isTight=False)
    mask_ele = mask_ele & looper_utils.mask_by_dR(events.Electron, photons_selected, 0.2)
    nEle = ak.num(events.Electron[mask_ele])
    
    mask_mu = event_selector.select_muon(events.Muon, isTight=False)
    mask_mu = mask_mu & looper_utils.mask_by_dR(events.Muon, photons_selected, 0.2)
    nMu = ak.num(events.Muon[mask_mu])
    
    sum_charge = (ak.sum(events.Tau[mask_tau].charge, axis=1) + ak.sum(events.Electron[mask_ele].charge, axis=1) + ak.sum(events.Muon[mask_mu].charge, axis=1) == 0)
    mask_dR_mll_0lep_2tau = looper_utils.mask_by_dR_mll_0lep_2tau(events.Tau[mask_tau], (0.2,3.5), (30,140))
    mask_dR_mll_1mu_1tau = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau], events.Muon[mask_mu], (0.3,3.5), (20,120))
    mask_dR_mll_1ele_1tau = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau], events.Electron[mask_ele], (0.3,3.5), (20,120))
    
    mask_dipho = (nPho == 2)
    mask_0lep_1tau = (nPho == 2) & (nTau == 1) & (nEle == 0) & (nMu == 0)
    mask_0lep_2tau = (nPho == 2) & (nTau == 2) & (nEle == 0) & (nMu == 0) & (sum_charge) & (mask_dR_mll_0lep_2tau)
    mask_2lep_0tau = (nPho == 2) & (nTau == 0) & (nEle + nMu == 2) & (sum_charge)
    mask_1lep_0tau = (nPho == 2) & (nTau == 0) & (nEle + nMu == 1)
    
    mask_1mu_1tau = (nPho == 2) & (nTau == 1) & (nEle == 0) & (nMu == 1) & (sum_charge) & (mask_dR_mll_1mu_1tau)
    mask_1ele_1tau = (nPho == 2) & (nTau == 1) & (nEle == 1) & (nMu == 0) & (sum_charge) & (mask_dR_mll_1ele_1tau)
    mask_1lep_1tau = (mask_1mu_1tau) | (mask_1ele_1tau)
    
    masks = {"dipho": mask_dipho,
             "0lep_1tau": mask_0lep_1tau,
             "0lep_2tau": mask_0lep_2tau,
             "1lep_1tau": mask_1lep_1tau,
             "2lep_0tau": mask_2lep_0tau,
             "1lep_0tau": mask_1lep_0tau}
    
    hists = {}
    
    for key, mask in masks.items():
        print (key)
        #hists[key] = process_event(genWeight[mask], evt_vars[mask])
        #print ("process diphoton")
        hists[key] =  make_plots.process_diphoton(events.Photon[mask], g_Hidx[mask], mgg[mask], events.genWeight[mask])
        #print ("process tau")
        hists[key].update( make_plots.process_tau(events.Tau[mask_tau][mask], events.genWeight[mask]))
        #print ("process muon")
        hists[key].update( make_plots.process_muon(events.Muon[mask_mu][mask], events.genWeight[mask]))
        #print ("process electron")
        hists[key].update( make_plots.process_electron(events.Electron[mask_ele][mask], events.genWeight[mask]))
        #print ("process tau-lep")
        if key == "1lep_1tau":
            tmphists = hists[key].update( make_plots.process_1tau_1lep(events.Tau[mask_tau][mask], events.Muon[mask_mu][mask],
                                                                       events.Electron[mask_ele][mask], events.genWeight[mask]))
            #if bool(tmphists):
            #    hists[key].update(tmphists)
    
    return hists

In [80]:
%%time
#hists = process(chunks["signal"][0])
#hists = process((chunks["signal"][0][0], 0, 200) )
hists = process(chunks["ZH"][0])

dipho
0lep_1tau
0lep_2tau
1lep_1tau
2lep_0tau
1lep_0tau
CPU times: user 30.1 s, sys: 1.3 s, total: 31.4 s
Wall time: 35.9 s


In [10]:
hists

{'dipho': {'pho_pT1': [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00    0.62 ± 0.06    5.16 ± 0.18
   15.85 ± 0.31   39.61 ± 0.49   59.44 ± 0.61   70.29 ± 0.66   74.45 ± 0.68
   71.27 ± 0.66   62.10 ± 0.62   53.47 ± 0.58   46.18 ± 0.53   36.68 ± 0.48
   30.34 ± 0.43   25.04 ± 0.39   19.73 ± 0.35   16.18 ± 0.32   85.55 ± 0.73],
  'pho_pT2': [ 0.00 ± 0.00    0.00 ± 0.00    1.90 ± 0.11   46.00 ± 0.53   153.34 ± 0.97
   139.98 ± 0.93   110.62 ± 0.83   79.65 ± 0.70   56.42 ± 0.59
   36.79 ± 0.48   25.75 ± 0.40   17.24 ± 0.33   11.88 ± 0.27    8.21 ± 0.23
    5.85 ± 0.19    4.29 ± 0.16    3.11 ± 0.14    2.27 ± 0.12    1.88 ± 0.11
    6.81 ± 0.21],
  'pho_pTom1': [ 0.00 ± 0.00    0.00 ± 0.00    0.04 ± 0.02    7.58 ± 0.22   39.27 ± 0.49
   84.67 ± 0.72   96.98 ± 0.77   97.53 ± 0.78   82.51 ± 0.71   66.69 ± 0.64
   52.69 ± 0.57   40.24 ± 0.50   30.73 ± 0.44   22.55 ± 0.37   18.05 ± 0.33
   13.57 ± 0.29   10.49 ± 0.25    8.61 ± 0.23   39.78 ± 0.50],
  'pho_pTom2': [ 0.00 ± 0.00    0.00 ± 0.00   33.

In [11]:
hists['1lep_1tau']['mtaumu']

[ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00    0.06 ± 0.02    0.80 ± 0.07
  0.99 ± 0.08    1.87 ± 0.11    2.12 ± 0.11    2.21 ± 0.12    1.81 ± 0.11
  1.79 ± 0.11    1.30 ± 0.09    0.95 ± 0.08    0.72 ± 0.07    0.46 ± 0.05
  0.19 ± 0.03    0.12 ± 0.03    0.07 ± 0.02    0.04 ± 0.02    0.01 ± 0.01
  0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00
  0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00]

In [12]:
hists['dipho']['pho_pT1'].integral
#hists['dipho']['pho_pT1'].integral_error

711.9703134298325

## send to dask

In [13]:
#from dask.distributed import Client
#client = Client('tcp://169.228.130.74:5608')
#client

In [14]:
from dask.distributed import Client
#client = Client(memory_limit='4GB', n_workers=32, threads_per_worker=1)
c = make_htcondor_cluster(memory="6GB",local = False)
c.scale(100)
client = Client(c)
client

Client Scheduler: tcp://169.228.130.74:7610 Dashboard: http://169.228.130.74:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [15]:
c

In [16]:
def g():
    import glob
    allpys = glob.glob("./*py")
    return allpys
g()

['./__init__.py',
 './cachepreload.py',
 './condor_utils.py',
 './event_selector.py',
 './loop.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py']

In [17]:
x = client.submit(g)

In [70]:
chunks["EGamma_2018D"]
#Remove this one file for now
for i in range(len(chunks["EGamma_2018D"])):
    if chunks["EGamma_2018D"][0][i] == "/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_367.root":
        del chunks["EGamma_2018D"][0][i]
        break

[['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_1.root',
  0,
  30073],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_10.root',
  0,
  30248],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_100.root',
  0,
  26172],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_101.root',
  0,
  37587],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_102.root',
  0,
  29479],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_103.root',
  0,
  29012],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_104.root',
  0,
  27905],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_105.root',
  0,
  33747],
 ['/hadoop/cms/store/user/legianni/ProjectMetis/EGamma_Run2018D____v4/test_nanoaodSkim_106.root',
 

In [20]:
client.gather(x)

['./cachepreload.py',
 './event_selector.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py']

In [22]:
%%time
from dask.distributed import as_completed
import collections, functools, operator 

futures = {}
results = {}

cat_keys = ["dipho", "0lep_1tau", "0lep_2tau", "1lep_1tau", "2lep_0tau", "1lep_0tau"]

for process_key in chunks:
    print ("start process {}".format(process_key))
    futures[process_key] = client.map(process, chunks[process_key], retries=5)
    #print(futures)
    results_local = []
    results[process_key] = {}
    ## do you really need this while?
    while len(results_local) < len(chunks[process_key]): 
        
        ac = as_completed(futures[process_key], with_results=True)
        for future, result in ac:
            results_local.append(result)
    
    ## merge histograms by the key
    for cat_key in cat_keys: 
        dicts = [results_local[i][cat_key] for i in range(len(results_local))]  
        counter = collections.Counter() 
        for d in dicts:  
            counter.update(d) 
        results[process_key][cat_key] = dict(counter) 

start process EGamma_2018A
start process EGamma_2018B
start process EGamma_2018C
start process EGamma_2018D
start process DYJets
start process ttbar
start process ZG
start process WG
start process GJets_HT40To100
start process GJets_HT100To200
start process GJets_HT200To400
start process GJets_HT400To600
start process GJets_HT600ToInf
start process QCD_pT30To40
start process QCD_pT40ToInf
start process Diphoton
start process ZH
start process VH
start process signal
CPU times: user 1min 23s, sys: 8.73 s, total: 1min 32s
Wall time: 6min 11s


## save to disk?

- foler1: cat (dipho, 0lep_1tau, 0lep_2tau...)
    + folder2: process (data, signal, ZG...)
        - folder3: hists (pT, eta, phi...)

In [ ]:
client.shutdown()

In [23]:
results.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'ZH', 'VH', 'signal'])

In [24]:
results['signal'].keys()

dict_keys(['dipho', '0lep_1tau', '0lep_2tau', '1lep_1tau', '2lep_0tau', '1lep_0tau'])

In [25]:
results['signal']['dipho'].keys()

dict_keys(['pho_pT1', 'pho_pT2', 'pho_pTom1', 'pho_pTom2', 'pho_eta1', 'pho_eta2', 'pho_phi1', 'pho_phi2', 'pho_id1', 'pho_id2', 'tau_pT1', 'tau_eta1', 'tau_phi1', 'tau_deeptau_vs_j_1', 'tau_deeptau_vs_m_1', 'tau_deeptau_vs_e_1', 'n_tau', 'mtautau', 'dR_tautau', 'tau_pT2', 'tau_eta2', 'tau_phi2', 'tau_deeptau_vs_j_2', 'tau_deeptau_vs_m_2', 'tau_deeptau_vs_e_2', 'muon_pT1', 'muon_eta1', 'muon_phi1', 'muon_iso1', 'n_muon', 'mmumu', 'dR_mumu', 'muon_pT2', 'muon_eta2', 'muon_phi2', 'muon_iso2', 'electron_pT1', 'electron_eta1', 'electron_phi1', 'electron_iso1', 'n_electron', 'mee', 'dR_ee', 'electron_pT2', 'electron_eta2', 'electron_phi2', 'electron_iso2'])

In [26]:
%%time
from subprocess import call
import json

process_keys = results.keys()
cat_keys = results['signal'].keys()

tag = "basic_dR_mll_cut"

for cat_key in cat_keys:
    for process_key in process_keys:
        dirname = './hists/' + tag + '/' + cat_key + '/' + process_key + '/'
        call('mkdir -p ' + dirname, shell=True)
        for hist_key in results[process_key][cat_key].keys():
            histname = dirname + hist_key +'_'+year+'_'+skim_version
            results[process_key][cat_key][hist_key].to_json(histname+".json")
            with open(histname, "w") as f:
                data = json.dump(results[process_key][cat_key][hist_key].to_json(histname+".json"), f)

CPU times: user 2.59 s, sys: 5.17 s, total: 7.76 s
Wall time: 12.8 s
